In [1]:
from sage.all import *
import numpy as np
import galois

In [2]:
def construct_hom_classical_code(num_bits: int, pc_mat: np.array) -> ChainComplex:
    """ 
    Construct chain complex for classical code from parity-check matrix

    Parameters:
    -----------
    num_bits - Number of bits of code
    pc_mat - parity-check matrix for code

    Returns:
    --------
    Chain complex representing code
    """
    # Define vector spaces for chain complex
    k_1 = len(pc_mat)
    C_0 = VectorSpace(GF(2), num_bits)
    C_1 = VectorSpace(GF(2), k_1)

    # Define boundary operators
    d_0 = matrix(pc_mat.tolist())

    # Define chain complex
    chain_complex_data = [C_0, d_0, C_1]
    #chain_complex_data = {1:d_1}
    code = ChainComplex(data=chain_complex_data, base_ring=GF(2), degree_of_differential=1)
    return code

def construct_hom_CSS_code(num_qubits: int, pc_mat_X: np.array, pc_mat_Z: np.array) -> ChainComplex:
    """ 
    Construct chain complex for CSS code from parity-check matrices

    Parameters:
    -----------
    num_qubits - Number of qubits of code
    pc_mat_X - X parity-check matrix
    pc_mat_Z - Z parity-check matrix

    Returns:
    --------
    Chain complex representing CSS code
    """

    # Define vector spaces for chain complex
    k_1 = len(pc_mat_Z)
    k_2 = len(pc_mat_X)
    C_0 = VectorSpace(GF(2), k_1)
    C_1 = VectorSpace(GF(2), num_qubits)
    C_2 = VectorSpace(GF(2), k_2)

    # Define boundary operators
    d_0 = matrix((pc_mat_Z.T).tolist())
    d_1 = matrix(pc_mat_X.tolist())

    # Define chain complex
    chain_complex_data = [C_0, d_0, C_1, d_1, C_2]
    #chain_complex_data = {1:d_1, 2:d_2}
    code = ChainComplex(data=chain_complex_data, base_ring=GF(2), degree_of_differential=1)
    return code

In [3]:
# Let us construct the classical repetition code as a chain complex
rep_code_mat = np.array([
    [1,1,0],
    [0,1,1],
    [1,0,1]
])

rep_code = construct_hom_classical_code(3, rep_code_mat)
ascii_art(rep_code)

            [1 1 0]      
            [0 1 1]      
            [1 0 1]      
 0 <-- C_1 <-------- C_0 <-- 0 

In [4]:
# Let us construct the Steane Code
x_pc_mat = np.array([[0,0,0,1,1,1,1], [0,1,1,0,0,1,1], [1,0,1,0,1,0,1]])
z_pc_mat = np.array([[0,0,0,1,1,1,1], [0,1,1,0,0,1,1], [1,0,1,0,1,0,1]])

steane_code = construct_hom_CSS_code(7, x_pc_mat, z_pc_mat)
ascii_art(steane_code)

                                  [0 0 1]      
                                  [0 1 0]      
                                  [0 1 1]      
                                  [1 0 0]      
            [0 0 0 1 1 1 1]       [1 0 1]      
            [0 1 1 0 0 1 1]       [1 1 0]      
            [1 0 1 0 1 0 1]       [1 1 1]      
 0 <-- C_2 <---------------- C_1 <-------- C_0 <-- 0 

In [5]:
# Let us try to construct the Toric code via a product of chain complexes of classical codes
mat = np.array([
    [1,0,1],
    [0,1,1],
    [1,0,1]
])

rep_code_1 = construct_hom_classical_code(3, mat)
rep_code_2 = construct_hom_classical_code(3, mat)
toric_code_complex = rep_code_1.tensor(rep_code_2)
ascii_art(toric_code_complex)

                                                        [1 0 1 0 0 0 0 0 0] 
                                                        [0 1 1 0 0 0 0 0 0] 
                                                        [1 0 1 0 0 0 0 0 0] 
                                                        [0 0 0 1 0 1 0 0 0] 
                                                        [0 0 0 0 1 1 0 0 0] 
                                                        [0 0 0 1 0 1 0 0 0] 
                                                        [0 0 0 0 0 0 1 0 1] 
                                                        [0 0 0 0 0 0 0 1 1] 
                                                        [0 0 0 0 0 0 1 0 1] 
            [1 0 0 0 0 0 1 0 0 1 0 1 0 0 0 0 0 0]       [1 0 0 0 0 0 1 0 0] 
            [0 1 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 0]       [0 1 0 0 0 0 0 1 0] 
            [0 0 1 0 0 0 0 0 1 1 0 1 0 0 0 0 0 0]       [0 0 1 0 0 0 0 0 1] 
            [0 0 0 1 0 0 1 0 0 0 0 0 1 0 1 0 0 0]       [0 0 0 1 0 0 1 0 0] 

In [6]:
# Relevant Boundary operators
d_0 = toric_code_complex.differential(0)
d_0_T = d_0.transpose()
d_1 = toric_code_complex.differential(1)
d_1_T = d_1.transpose()

# Kernel and Image of boundary operators to compute logial operators
ker_d_1 = d_1.right_kernel()
im_d_1_T = d_1_T.transpose().image()
im_d_0 = d_0.transpose().image()
ker_d_0_T = d_0.right_kernel()

# Dimension of code
code_dim = (ker_d_1/im_d_0).dimension()

# Logical Z operators
ker_d_1_vecs = [v for v in ker_d_1]
im_d_0_vecs = [v for v in im_d_0]
log_Z_ops = [v for v in ker_d_1_vecs if v not in im_d_0_vecs]

# Logical X operators
ker_d_0_T_vecs = [v for v in ker_d_0_T]
im_d_1_T_vecs = [v for v in im_d_1_T]
log_X_ops = [v for v in ker_d_0_T_vecs if v not in im_d_1_T_vecs]

print("The dimension of the toric code is: " + str(code_dim))


The dimension of the toric code is: 2


In [7]:
dual_toric = toric_code_complex.dual()
ascii_art(dual_toric)

                                                        [1 0 0 0 0 0 1 0 0] 
                                                        [0 1 0 0 0 0 0 1 0] 
                                                        [0 0 1 0 0 0 0 0 1] 
                                                        [0 0 0 1 0 0 0 0 0] 
                                                        [0 0 0 0 1 0 0 0 0] 
                                                        [0 0 0 0 0 1 0 0 0] 
                                                        [1 0 0 1 0 0 1 0 0] 
                                                        [0 1 0 0 1 0 0 1 0] 
                                                        [0 0 1 0 0 1 0 0 1] 
            [1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0]       [1 0 1 0 0 0 0 0 0] 
            [0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0]       [0 1 0 0 0 0 0 0 0] 
            [1 1 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1]       [1 1 1 0 0 0 0 0 0] 
            [0 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0]       [0 0 0 1 0 1 0 0 0] 

In [12]:
dual_toric.homology()

[
(1)
]